# NYC Apartment Search

_[Project prompt](https://docs.google.com/document/d/1BYVyFBDcTywdUlanH0ysfOrNWPgl7UkqXA7NeewTzxA/edit#heading=h.bpxu7uvknnbk)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an idea of a possible approach.**_

## Setup

In [45]:
# All import statements needed for the project, for example:

import json
import pathlib
import urllib.parse

import pandas as pd
import geoalchemy2 as gdb
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import requests
import shapely
import sqlalchemy as db

from sqlalchemy.orm import declarative_base

In [46]:
# Any constants you might need; some have been added for you

# Where data files will be read from/written to - this should already exist
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

NYC_DATA_APP_TOKEN = "UydgE9GUfZyuG9IpbKml1aKct"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

DB_NAME = "IE4501_Project_DB"
DB_USER = "postgres"
DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"
# directory where DB queries for Part 3 will be saved
QUERY_DIR = pathlib.Path("queries")

In [47]:
# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

## Part 1: Data Preprocessing

In [48]:
def download_nyc_geojson_data(url, force=False):
    parsed_url = urllib.parse.urlparse(url)
    url_path = parsed_url.path.strip("/")
    
    filename = DATA_DIR / url_path
    
    if force or not filename.exists():
        print(f"Downloading {url} to {filename}...")
        
        ...
        
        with open(filename, "w") as f:
            json.dump(..., f)
        print(f"Done downloading {url}.")

    else:
        print(f"Reading from {filename}...")

    return filename

In [49]:
def load_and_clean_zipcodes(zipcode_datafile):
     # Load the data using geopandas
    gdf = gpd.read_file(ZIPCODE_DATA_FILE)
    
    print(gdf)

    # Drop unnecessary columns. Here, you need to decide which columns are relevant
    # For example, if you only need the zipcode and geometry, you can drop the rest
    columns_to_keep = ['geometry']
    gdf = gdf[columns_to_keep]

    # Remove invalid data points
    # This depends on what you define as invalid. As an example, you might want to remove rows with missing values
    gdf = gdf.dropna()

    # Normalize column names
    # If you want to make column names lowercase for consistency
    gdf.columns = [col.lower() for col in gdf.columns]

    # Normalize data types if needed
    # For example, ensuring ZIPCODE is a string, not a number
    gdf['geometry'] = gdf['geometry'].astype(str)

    # Additional cleaning steps can be added here depending on your specific requirements

    return gdf
    #raise NotImplementedError()

In [50]:
load_and_clean_zipcodes(ZIPCODE_DATA_FILE)

                                              geometry
0    POLYGON ((1038098.252 188138.380, 1038141.936 ...
1    POLYGON ((1001613.713 186926.440, 1002314.243 ...
2    POLYGON ((1011174.276 183696.338, 1011373.584 ...
3    POLYGON ((995908.365 183617.613, 996522.848 18...
4    POLYGON ((991997.113 176307.496, 992042.798 17...
..                                                 ...
258  POLYGON ((950767.507 172848.969, 950787.510 17...
259  POLYGON ((1028453.995 167153.410, 1027813.010 ...
260  POLYGON ((995877.318 203206.075, 995968.511 20...
261  POLYGON ((997731.761 219560.922, 997641.948 21...
262  POLYGON ((986038.661 213051.063, 986135.314 21...

[263 rows x 1 columns]


/var/folders/1g/fy105j8s3xqf2l38dtzk0kcr0000gn/T/ipykernel_48959/339113466.py:22: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].astype(str)


,geometry
0,"POLYGON ((1038098.251871 188138.380007, 103814..."
1,"POLYGON ((1001613.712964 186926.439517, 100231..."
2,"POLYGON ((1011174.275536 183696.33771, 1011373..."
3,"POLYGON ((995908.365451 183617.612802, 996522...."
4,"POLYGON ((991997.113431 176307.49586, 992042.7..."
...,...
258,"POLYGON ((950767.506586 172848.968656, 950787...."
259,"POLYGON ((1028453.994913 167153.409838, 102781..."
260,"POLYGON ((995877.318269 203206.074937, 995968...."
261,"POLYGON ((997731.760754 219560.922148, 997641...."


In [51]:
def download_and_clean_311_data():
    import requests

# Your Application Token
    app_token = 'UydgE9GUfZyuG9IpbKml1aKct'

# API endpoints for 311 and tree data
    api_endpoint_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json?$limit=5000'


# Headers for authentication
    headers = {
        'X-App-Token': app_token
    }

# Make a GET request to the 311 data endpoint
    response_311 = requests.get(api_endpoint_311, headers=headers)
    data_311 = response_311.json()

# Assuming you have loaded data into DataFrame
    df = pd.DataFrame(data_311)  # Example for 311 data

# Removing unnecessary columns
    columns_to_keep = ['created_date', 'incident_zip', 'latitude','longitude',':@computed_region_efsh_h5xi']  # replace with actual column names
    df = df[columns_to_keep]

# Remove invalid data points
    #df = df[df[':@computed_region_efsh_h5xi'] != null] 
    df = df.dropna()# Replace with your criteria

# Normalize column names
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]

# Convert data types
    #df['date_column'] = pd.to_datetime(df['date_column'])
   # df['numeric_column'] = pd.to_numeric(df['numeric_column'], errors='coerce')

# For geospatial data
    gdf311 = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    gdf311.set_crs("EPSG:4326", inplace=True)  # Replace with current SRID
    return gdf311
    sampled_df = df.sample(n=10)  # Replace 100 with the number of samples you need
    print(sampled_df)


In [52]:
download_and_clean_311_data()

,created_date,incident_zip,latitude,longitude,:@computed_region_efsh_h5xi,geometry
0,2023-12-01T12:00:00.000,10465,40.84384365721464,-73.82454971742995,11274,POINT (-73.82455 40.84384)
1,2023-12-01T01:03:37.000,10308,40.56203453710598,-74.15721538110013,10695,POINT (-74.15722 40.56203)
2,2023-12-01T01:03:29.000,11207,40.68471967420527,-73.90681258883566,17214,POINT (-73.90681 40.68472)
3,2023-12-01T01:03:27.000,10467,40.87631257939169,-73.86697759703779,11605,POINT (-73.86698 40.87631)
4,2023-12-01T01:03:20.000,10024,40.78515581987176,-73.97900165965613,12421,POINT (-73.97900 40.78516)
...,...,...,...,...,...,...
4995,2023-11-30T13:36:08.000,10027,40.81688523235436,-73.95252798722301,12424,POINT (-73.95253 40.81689)
4996,2023-11-30T13:35:56.000,11213,40.6715931103321,-73.93288727278977,17615,POINT (-73.93289 40.67159)
4997,2023-11-30T13:35:47.000,10065,40.76327213006063,-73.95899539382252,10090,POINT (-73.95900 40.76327)
4998,2023-11-30T13:35:42.000,11373,40.7438331646918,-73.87706534371694,14784,POINT (-73.87707 40.74383)


In [53]:
def download_and_clean_tree_data():
    app_token = 'UydgE9GUfZyuG9IpbKml1aKct'
    api_endpoint_tree = 'https://data.cityofnewyork.us/resource/5rq2-4hqu.json?$limit=5000'
    headers = {
        'X-App-Token': app_token
    }
    response_tree = requests.get(api_endpoint_tree, headers=headers)
    data_tree = response_tree.json()
    df2 = pd.DataFrame(data_tree)
    gdf = gpd.GeoDataFrame(df2, geometry=gpd.points_from_xy(df2.longitude.astype(float), df2.latitude.astype(float)))
    gdf.set_crs("EPSG:4326", inplace=True)
    return gdf
    sample_tree = gdf.sample(n=10)
    print(sample_tree)
    

In [54]:
download_and_clean_tree_data()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,geometry
0,08/27/2015,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",3,0,OnCurb,Alive,Fair,Acer rubrum,...,16,QN17,Forest Hills,4073900,New York,40.72309177,-73.84421522,1027431.14821,202756.768749,POINT (-73.84422 40.72309)
1,09/03/2015,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",21,0,OnCurb,Alive,Fair,Quercus palustris,...,11,QN49,Whitestone,4097300,New York,40.79411067,-73.81867946,1034455.70109,228644.837379,POINT (-73.81868 40.79411)
2,09/05/2015,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,18,BK90,East Williamsburg,3044900,New York,40.71758074,-73.9366077,1001822.83131,200716.891267,POINT (-73.93661 40.71758)
3,09/05/2015,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,18,BK90,East Williamsburg,3044900,New York,40.71353749,-73.93445616,1002420.35833,199244.253136,POINT (-73.93446 40.71354)
4,08/30/2015,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",21,0,OnCurb,Alive,Good,Tilia americana,...,21,BK37,Park Slope-Gowanus,3016500,New York,40.66677776,-73.97597938,990913.775046,182202.425999,POINT (-73.97598 40.66678)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,08/30/2015,189566,223043,"{'type': 'Point', 'coordinates': [-73.97608466...",16,0,OnCurb,Alive,Poor,Platanus x acerifolia,...,21,BK37,Park Slope-Gowanus,3016500,New York,40.66682867,-73.97608467,990884.562834,182220.968285,POINT (-73.97608 40.66683)
4996,09/02/2015,196084,310272,"{'type': 'Point', 'coordinates': [-73.82801543...",29,0,OnCurb,Alive,Good,Platanus x acerifolia,...,14,QN60,Kew Gardens,4014000,New York,40.70452153,-73.82801544,1031934.75698,195999.476817,POINT (-73.82802 40.70452)
4997,08/26/2015,176319,338642,"{'type': 'Point', 'coordinates': [-73.78844286...",5,0,OnCurb,Alive,Good,Koelreuteria paniculata,...,10,QN12,Hammels-Arverne-Edgemere,4097204,New York,40.59284143,-73.78844287,1043004.96811,155335.48203,POINT (-73.78844 40.59284)
4998,08/25/2015,172943,228140,"{'type': 'Point', 'coordinates': [-73.99084297...",12,0,OnCurb,Alive,Good,Quercus shumardii,...,17,BK88,Borough Park,3023800,New York,40.62945401,-73.99084297,986791.755075,168603.579509,POINT (-73.99084 40.62945)


In [55]:
def load_and_clean_zillow_data():
   # Load the data using pandas, as we need to handle both non-spatial and spatial data
    df_zillow = pd.read_csv(ZILLOW_DATA_FILE)

    # Keep only the necessary columns
    columns_to_keep = ["RegionID", "SizeRank", "RegionName", "StateName", "City", "Metro", "CountyName", "2023-01-31", "2023-08-31"]
    df_zillow = df_zillow[columns_to_keep]

    # Remove rows with missing values in any of the columns
    df_zillow = df_zillow.dropna()
    df_zillow = df_zillow[df_zillow['City']=='New York']

    # Normalize column names to lowercase
    df_zillow.columns = [col.lower() for col in df_zillow.columns]

    # Normalize data types
    # Assuming 'RegionID' and 'SizeRank' should be integers, and dates should be in datetime format
    df_zillow['regionid'] = df_zillow['regionid'].astype(int)
    df_zillow['sizerank'] = df_zillow['sizerank'].astype(int)


    # Convert DataFrame to GeoDataFrame
    gdf = gpd.GeoDataFrame(df_zillow)

    # Normalize SRID of any geometry if present
    # Example: gdf.set_crs(epsg=YOUR_EPSG_CODE, inplace=True)

    return gdf

In [56]:
load_and_clean_zillow_data()

,regionid,sizerank,regionname,statename,city,metro,countyname,2023-01-31,2023-08-31
4,62093,7,11385,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2895.699421,3064.476503
6,62019,9,11208,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,2588.030194,2737.547470
13,61807,17,10467,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Bronx County,2155.617718,2353.686402
14,62085,18,11373,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2255.604528,2302.557354
15,62037,19,11226,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,2680.683700,2785.320137
...,...,...,...,...,...,...,...,...,...
6441,61771,10844,10280,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,4059.897856,4602.724208
6550,61773,12178,10282,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,7143.347680,7444.236260
6561,62010,12258,11109,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,4159.759749,4490.367074
6644,61620,14713,10006,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",New York County,3747.416799,4042.831474


In [63]:

geodf_zipcode_data = load_and_clean_zipcodes(ZIPCODE_DATA_FILE)
geodf_311_data = download_and_clean_311_data()
geodf_tree_data = download_and_clean_tree_data()
df_zillow_data = load_and_clean_zillow_data()


                                              geometry
0    POLYGON ((1038098.252 188138.380, 1038141.936 ...
1    POLYGON ((1001613.713 186926.440, 1002314.243 ...
2    POLYGON ((1011174.276 183696.338, 1011373.584 ...
3    POLYGON ((995908.365 183617.613, 996522.848 18...
4    POLYGON ((991997.113 176307.496, 992042.798 17...
..                                                 ...
258  POLYGON ((950767.507 172848.969, 950787.510 17...
259  POLYGON ((1028453.995 167153.410, 1027813.010 ...
260  POLYGON ((995877.318 203206.075, 995968.511 20...
261  POLYGON ((997731.761 219560.922, 997641.948 21...
262  POLYGON ((986038.661 213051.063, 986135.314 21...

[263 rows x 1 columns]


/var/folders/1g/fy105j8s3xqf2l38dtzk0kcr0000gn/T/ipykernel_48959/339113466.py:22: UserWarning: Geometry column does not contain geometry.
  gdf['geometry'] = gdf['geometry'].astype(str)


In [64]:
print(type(geodf_311_data))
print(type(geodf_tree_data))

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [59]:
# Show basic info about each dataframe
geodf_zipcode_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   geometry  263 non-null    object
dtypes: object(1)
memory usage: 2.2+ KB


In [60]:
# Show first 5 entries about each dataframe
geodf_zipcode_data.head()

,geometry
0,"POLYGON ((1038098.251871 188138.380007, 103814..."
1,"POLYGON ((1001613.712964 186926.439517, 100231..."
2,"POLYGON ((1011174.275536 183696.33771, 1011373..."
3,"POLYGON ((995908.365451 183617.612802, 996522...."
4,"POLYGON ((991997.113431 176307.49586, 992042.7..."


In [65]:
geodf_311_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4908 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   created_date                 4908 non-null   object  
 1   incident_zip                 4908 non-null   object  
 2   latitude                     4908 non-null   object  
 3   longitude                    4908 non-null   object  
 4   :@computed_region_efsh_h5xi  4908 non-null   object  
 5   geometry                     4908 non-null   geometry
dtypes: geometry(1), object(5)
memory usage: 268.4+ KB


In [66]:
geodf_311_data.head()

,created_date,incident_zip,latitude,longitude,:@computed_region_efsh_h5xi,geometry
0,2023-12-01T12:00:00.000,10465,40.84384365721464,-73.82454971742995,11274,POINT (-73.82455 40.84384)
1,2023-12-01T01:03:37.000,10308,40.56203453710598,-74.15721538110013,10695,POINT (-74.15722 40.56203)
2,2023-12-01T01:03:29.000,11207,40.68471967420527,-73.90681258883566,17214,POINT (-73.90681 40.68472)
3,2023-12-01T01:03:27.000,10467,40.87631257939169,-73.86697759703779,11605,POINT (-73.86698 40.87631)
4,2023-12-01T01:03:20.000,10024,40.78515581987176,-73.97900165965613,12421,POINT (-73.97900 40.78516)


In [67]:
geodf_tree_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   created_at  5000 non-null   object  
 1   tree_id     5000 non-null   object  
 2   block_id    5000 non-null   object  
 3   the_geom    5000 non-null   object  
 4   tree_dbh    5000 non-null   object  
 5   stump_diam  5000 non-null   object  
 6   curb_loc    5000 non-null   object  
 7   status      5000 non-null   object  
 8   health      4810 non-null   object  
 9   spc_latin   4810 non-null   object  
 10  spc_common  4810 non-null   object  
 11  steward     4810 non-null   object  
 12  guards      4810 non-null   object  
 13  sidewalk    4810 non-null   object  
 14  user_type   5000 non-null   object  
 15  problems    4810 non-null   object  
 16  root_stone  5000 non-null   object  
 17  root_grate  5000 non-null   object  
 18  root_other  5000 non-null   object  
 19

In [68]:
geodf_tree_data.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp,geometry
0,08/27/2015,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",3,0,OnCurb,Alive,Fair,Acer rubrum,...,16,QN17,Forest Hills,4073900,New York,40.72309177,-73.84421522,1027431.14821,202756.768749,POINT (-73.84422 40.72309)
1,09/03/2015,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",21,0,OnCurb,Alive,Fair,Quercus palustris,...,11,QN49,Whitestone,4097300,New York,40.79411067,-73.81867946,1034455.70109,228644.837379,POINT (-73.81868 40.79411)
2,09/05/2015,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,18,BK90,East Williamsburg,3044900,New York,40.71758074,-73.9366077,1001822.83131,200716.891267,POINT (-73.93661 40.71758)
3,09/05/2015,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,18,BK90,East Williamsburg,3044900,New York,40.71353749,-73.93445616,1002420.35833,199244.253136,POINT (-73.93446 40.71354)
4,08/30/2015,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",21,0,OnCurb,Alive,Good,Tilia americana,...,21,BK37,Park Slope-Gowanus,3016500,New York,40.66677776,-73.97597938,990913.775046,182202.425999,POINT (-73.97598 40.66678)


In [ ]:
df_zillow_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 4771 entries, 0 to 6721
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   regionid    4771 non-null   int64  
 1   sizerank    4771 non-null   int64  
 2   regionname  4771 non-null   int64  
 3   statename   4771 non-null   object 
 4   city        4771 non-null   object 
 5   metro       4771 non-null   object 
 6   countyname  4771 non-null   object 
 7   2023-01-31  4771 non-null   float64
 8   2023-08-31  4771 non-null   float64
dtypes: float64(2), int64(3), object(4)
memory usage: 372.7+ KB


In [69]:
df_zillow_data.head()

,regionid,sizerank,regionname,statename,city,metro,countyname,2023-01-31,2023-08-31
4,62093,7,11385,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2895.699421,3064.476503
6,62019,9,11208,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,2588.030194,2737.547470
13,61807,17,10467,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Bronx County,2155.617718,2353.686402
14,62085,18,11373,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,2255.604528,2302.557354
15,62037,19,11226,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Kings County,2680.683700,2785.320137


## Part 2: Storing Data

In [ ]:
def setup_new_postgis_database(username, db_name):
    

SyntaxError: incomplete input (675149776.py, line 2)

In [ ]:
!createdb IE4501_Project_DB

createdb: error: database creation failed: ERROR:  database "IE4501_Project_DB" already exists


In [ ]:
!psql --dbname IE4501_Project_DB -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


In [ ]:
engine = db.create_engine(DB_URL)

NoSuchModuleError: Can't load plugin: sqlalchemy.dialects:postgres.psycopg2

In [ ]:
setup_new_postgis_database(DB_USER, DB_NAME)

### Creating Tables


These are just a couple of options to creating your tables; you can use one or the other, a different method, or a combination.

In [ ]:
engine = db.create_engine(DB_URL)

#### Option 1: SQL

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the SQL statements to create your 4 tables
ZIPCODE_SCHEMA = """
TODO
"""

NYC_311_SCHEMA = """
TODO
"""

NYC_TREE_SCHEMA = """
TODO
"""

ZILLOW_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DB_SCHEMA_FILE, "w") as f:
    f.write(ZIPCODE_SCHEMA)
    f.write(NYC_311_SCHEMA)
    f.write(NYC_TREE_SCHEMA)
    f.write(ZILLOW_SCHEMA)

In [ ]:
# If using SQL (as opposed to SQLAlchemy), execute the schema files to create tables
with engine.connect() as connection:
    pass

#### Option 2: SQLAlchemy

In [ ]:
Base = declarative_base()

class Tree(Base):
    __tablename__ = "trees"

    ...


In [ ]:
Base.metadata.create_all(engine)

### Add Data to Database

These are just a couple of options to write data to your tables; you can use one or the other, a different method, or a combination.

#### Option 1: SQL

In [ ]:
def write_dataframes_to_table(tablename_to_dataframe):
    # write INSERT statements or use pandas/geopandas to write SQL
    raise NotImplemented()

In [ ]:
tablename_to_dataframe = {
    "zipcodes": geodf_zipcode_data,
    "complaints": geodf_311_data,
    "trees": geodf_tree_data,
    "rents": df_zillow_data,
}

In [ ]:
write_dataframes_to_table(tablename_to_dataframe)

#### Option 2: SQLAlchemy

In [ ]:
Session = db.orm.sessionmaker(bind=engine)
session = Session()

In [ ]:
for row in geodf_tree_data.iterrows():
    tree = Tree(...)
    session.add(tree)

In [ ]:
session.commit()

## Part 3: Understanding the Data

### Query 1

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

In [ ]:
QUERY_1_FILENAME = QUERY_DIR / "FILL_ME_IN"

QUERY_1 = """
FILL_ME_IN
"""

In [ ]:
with engine.connect() as conn:
    result = conn.execute(db.text(QUERY_1))
    for row in result:
        print(row)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query your database for the data needed.
    # You can put the data queried into a pandas/geopandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)